In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
print(os.getcwd())

/kaggle/working


In [4]:
%cd ..

/kaggle


In [5]:
import csv
data = {}
with open('./input/state-farm-distracted-driver-detection/driver_imgs_list.csv') as f:
  reader = csv.reader(f)
  next(reader)

  for row in reader:
    key = row[1].lower()
    if key in data:
      data[key].append(row[2])
    else: 
     data[key] = [row[2]]

In [6]:
class_names_list = list(data.keys())
class_names_list

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [11]:

os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')
for class_name in class_names_list:
  os.mkdir(os.path.join('master_data/training',class_name))
  os.mkdir(os.path.join('master_data/testing',class_name))

FileExistsError: [Errno 17] File exists: 'master_data'

In [12]:
print(os.listdir())

['lib', 'input', 'master_data', 'working']


In [15]:
print(os.listdir("./master_data/testing"))

['c1', 'c3', 'c5', 'c4', 'c9', 'c0', 'c6', 'c2', 'c8', 'c7']


In [17]:
from shutil import copyfile

In [19]:
split_size = 0.8

for class_name in class_names_list:
    class_length = len(os.listdir(os.path.join("./input/state-farm-distracted-driver-detection/imgs/train/",class_name)))
    file_name_list = os.listdir(os.path.join("./input/state-farm-distracted-driver-detection/imgs/train/",class_name))
    
    train_size = int(split_size * class_length)
    train_images = file_name_list[:train_size]
    test_images = file_name_list[train_size:]
    
    for image in train_images:
        source = os.path.join('./input/state-farm-distracted-driver-detection/imgs/train/',class_name,image)
        dest = os.path.join('./master_data/training',class_name,image)
        copyfile(source,dest)
        
    for image in test_images:
        source = os.path.join('./input/state-farm-distracted-driver-detection/imgs/train/',class_name,image)
        dest = os.path.join('./master_data/testing',class_name,image)
        copyfile(source,dest)
        
    
#     print(class_length)
#     print(len(train_images))
#     print(len(test_images))
#     print()
    

In [20]:
for class_name in class_names_list:
    class_length_train = len(os.listdir(os.path.join('./master_data/training',class_name)))
    class_length_test = len(os.listdir(os.path.join('./master_data/testing',class_name)))
    class_length = len(os.listdir(os.path.join("./input/state-farm-distracted-driver-detection/imgs/train/",class_name)))
    print(class_length_train)
    print(class_length_test)
    print(class_length)
    print()

1991
498
2489

1813
454
2267

1853
464
2317

1876
470
2346

1860
466
2326

1849
463
2312

1860
465
2325

1601
401
2002

1528
383
1911

1703
426
2129



In [21]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet import ResNet50
from keras.applications.resnet import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [22]:
IMAGE_SIZE = [224, 224]
train_path = './working/master_data/training'
valid_path = './working/master_data/testing'

In [23]:
res50 = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94773248/94765736 [==============================] - 2s 0us/step


In [24]:
for layer in res50.layers:
  layer.trainable = False

In [25]:
folders = glob('./master_data/training/*')
folders

['./master_data/training/c1',
 './master_data/training/c3',
 './master_data/training/c5',
 './master_data/training/c4',
 './master_data/training/c9',
 './master_data/training/c0',
 './master_data/training/c6',
 './master_data/training/c2',
 './master_data/training/c8',
 './master_data/training/c7']

In [26]:
x = Flatten()(res50.output)

In [27]:
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=res50.input, outputs=prediction)

In [28]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [29]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('./master_data/training',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('./master_data/testing',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_accuracy',patience=2,min_delta=0.01)

In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  callbacks = [es]
)

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')